In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
home_path = Path.home() / "Downloads"

In [3]:
data = pd.read_csv(home_path / "Data.csv")

In [4]:
data.drop(columns=['Country/Series-specific Notes', 'Estimates Start After'], inplace=True)

In [5]:
data.head()

,Country,Subject Descriptor,Units,Scale,2017,2018,2019,2020,2021,2022,2023,2024
0,Afghanistan,"Gross domestic product, constant prices",National currency,Billions,506.215,517.858,533.394,552.063,574.127,599.933,629.88,664.452
1,Afghanistan,"Inflation, average consumer prices",Index,NaN,110.998,111.693,113.815,117.799,123.099,129.254,135.717,142.503
2,Afghanistan,Unemployment rate,Percent of total labor force,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,"Gross domestic product, constant prices",National currency,Billions,789.653,822.522,852.66,885.573,919.717,956.167,994.155,"1,033.83"
4,Albania,"Inflation, average consumer prices",Index,NaN,103.295,105.39,107.498,110.077,113.16,116.554,120.051,123.653


In [6]:
#Ensuring the values are the same for all columns to get rid of them
print(f"\n{data['Scale'].value_counts()}")

print(f"\n{data['Units'].value_counts()}")


Billions    194
Name: Scale, dtype: int64

Index                           194
Percent of total labor force    194
National currency               194
Name: Units, dtype: int64


In [7]:
#id_vars are the variables we don't want to melt
#Unless explicitly stated, all the other variables in the dataframe will be melted,
# so they should all be of the same "type" (normally columns by year)
#Var_name provides the name of the newly formed column
#Value_name provides the name of the second column with the actual values
melted_data = pd.melt(data, id_vars=['Country', 'Subject Descriptor', 'Units', 'Scale'],
                       var_name="Year", value_name="Value")

In [8]:
melted_data.head()

,Country,Subject Descriptor,Units,Scale,Year,Value
0,Afghanistan,"Gross domestic product, constant prices",National currency,Billions,2017,506.215
1,Afghanistan,"Inflation, average consumer prices",Index,NaN,2017,110.998
2,Afghanistan,Unemployment rate,Percent of total labor force,NaN,2017,NaN
3,Albania,"Gross domestic product, constant prices",National currency,Billions,2017,789.653
4,Albania,"Inflation, average consumer prices",Index,NaN,2017,103.295


In [9]:
#Now that we've "melted" our year columns into one column, we can now pivot to something else.
#To break out the different measures shown for each country, we use pd.pivot_table
# This is a more generalizable version of pd.pivot, allowing us to use multiple indexes.
# We also set the aggregation function to first since we don't want mean values in the case of multiple
# values but simply to use the 'first' value found.
df = pd.DataFrame(pd.pivot_table(melted_data, index=['Country','Year'], columns='Subject Descriptor',
              values='Value', aggfunc='first'))

In [10]:
df.reset_index(inplace=True)

In [11]:
#Adding Back in Units and Scale Columns by simply adding to column names
df.columns = ['Country', 'Year','GDP (constant prices, national currency, Billions)',
              'INFLATION (average consumer prices, Index, NaN)', 
              'UNEMPLOYMENT RATE (percent of total work force, NaN)']

In [12]:
df.head(30)

,Country,Year,"GDP (constant prices, national currency, Billions)","INFLATION (average consumer prices, Index, NaN)","UNEMPLOYMENT RATE (percent of total work force, NaN)"
0,Afghanistan,2017,506.215,110.998,NaN
1,Afghanistan,2018,517.858,111.693,NaN
2,Afghanistan,2019,533.394,113.815,NaN
3,Afghanistan,2020,552.063,117.799,NaN
4,Afghanistan,2021,574.127,123.099,NaN
5,Afghanistan,2022,599.933,129.254,NaN
6,Afghanistan,2023,629.88,135.717,NaN
7,Afghanistan,2024,664.452,142.503,NaN
8,Albania,2017,789.653,103.295,13.756
9,Albania,2018,822.522,105.39,12.2


In [13]:
df.to_csv("Clean_Data.csv", index=False)